## 1. 코드 재활용을 위한 아발로니 파일 실행시키기

In [2]:
%run ../chap01/abalone.ipynb

## 2. 메인 함수 정의

In [3]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

## 3. 데이터 적재 함수의 정의

In [5]:
def load_pulsar_dataset():
    with open('../../data/chap02/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        row = []
        for row in csvreader:
            rows.append(row)
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32')

## 4. 후처리 과정에 대한 순전파와 역전파 함수의 재정의

In [6]:
def forward_postproc(output, y):
    entropy = simgmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

## 5. 정확도 계산 함수의 재정의

In [7]:
def eval_accuracy(output, y):
    estimate = np.greater(output, 0) # output의 shape대로 0보다 큰 원소만 True, 나머지는 False 반환
    answer = np.greater(y, 0.5)
    correct = np.equal(estimate, answer) # estimate와 answer의 shape대로 원소 각각을 비교해 동일할 때 True, 다르면 False 반환
    
    return np.mean(correct)  # 정답(1)의 비율 반환

## 6. 시그모이드 관련 함수 정의

In [12]:
def relu(x):
    return np.maximum(x, 0) # max가 아니라 maximum을 사용해야 0보다 작은 부분을 대치함

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def sigmoid_derv(x, y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)